In [31]:
import json
from pathlib import Path

import duckdb
import boto3
import plotly.express as px
import polars as pl
import s3fs

In [32]:
with open(Path("creds/creds.json"), "r") as target:
    creds = json.load(target)
    storage_options = creds["AWS"]


In [ ]:
a = set(list(storage_options.keys()))
b = set(["aws_secret_access_keya", "aws_access_key_id", "aws_region"])

b <= a

In [ ]:
# pull gzip data from s3
s3 = s3fs.S3FileSystem(
    key=storage_options["aws_access_key_id"],
    secret=storage_options["aws_secret_access_key"],
)


filepath = f"s3://{storage_options['aws_bucket']}/datalog/{distrik}/{deviceid}/{hivehour}/{hivehour}"
bucketpath


In [ ]:
s3paths = s3.glob(filepath)
s3paths = [i.replace("smartdbucket/datalog/BRCG/SLS30I009/", "") for i in s3paths]

In [ ]:
# duckdb s3 connection config
conn = duckdb.connect()

conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

conn.execute(f"SET s3_region = '{storage_options['aws_region']}';")
conn.execute(f"SET s3_access_key_id = '{storage_options['aws_access_key_id']}';")
conn.execute(
    f"SET s3_secret_access_key = '{storage_options['aws_secret_access_key']}';"
)


In [ ]:
list_unit = [
    ("SLS30I009", "LD780"),
    ("SLS30I015", "LD781"),
    ("SLS30I016", "LD782"),
    ("SLS30I039", "LD776"),
    ("SLS30I146", "LD772"),
    ("SLS30I477", "LD778"),
    ("SDLIR038", "LD921"),
    ("SLS30I623", "LD783"),
]

In [ ]:
df = conn.sql(f"""
SELECT CAST(to_timestamp(heartbeat) as DATE)as tgl_utc,COUNT(1) as jml
FROM read_json_auto('{bucketpath}')
GROUP BY CAST(to_timestamp(heartbeat) as DATE)
ORDER BY CAST(to_timestamp(heartbeat) as DATE)
           """).pl()

df

In [ ]:
list_unit

In [22]:
distrik = "BRCG"
deviceid = "SLS30I009"


def count_canbus_rows(conn, devid, hivehour):
    bucketpath = f"s3://{storage_options['aws_bucket']}/datalog/{distrik}/{devid}/{hivehour}/{hivehour}.txt.gz"
    df = conn.sql(f"""
SELECT CAST(to_timestamp(heartbeat) as DATE)+ INTERVAL 8 HOUR as datetime_wita,heartbeat,unitno,deviceid,gpsspeed,gpsnumsat,vehiclespeed,speedsource,
FROM read_json_auto('{bucketpath}')
WHERE gpsspeed > 0 AND VehicleSpeed > 0
           """).pl()
    print(devid, "unit has :", len(df), "rows")
    return df

In [ ]:
list_unit[1][1]

In [23]:
del main_df

NameError: name 'main_df' is not defined

In [ ]:
hivehour = "2025121*"
with duckdb.connect() as conn:
    conn = duckdb.connect()

    conn.execute("INSTALL httpfs;")
    conn.execute("LOAD httpfs;")

    conn.execute(f"SET s3_region = '{storage_options['aws_region']}';")
    conn.execute(f"SET s3_access_key_id = '{storage_options['aws_access_key_id']}';")
    conn.execute(
        f"SET s3_secret_access_key = '{storage_options['aws_secret_access_key']}';"
    )

    for sls, unitno in list_unit:
        try:
            if main_df is not None:
                df = count_canbus_rows(conn, sls, hivehour)
                main_df = pl.concat([main_df, df])
        except NameError:
            # Code to execute if 'df' variable has not been assigned
            main_df = count_canbus_rows(conn, sls, hivehour)
            print("DataFrame variable is not defined.")

SLS30I009 unit has : 72595 rows
DataFrame variable is not defined.
SLS30I015 unit has : 61755 rows
SLS30I016 unit has : 73793 rows
SLS30I039 unit has : 119338 rows
SLS30I146 unit has : 72103 rows
SLS30I477 unit has : 40564 rows
SDLIR038 unit has : 82489 rows
SLS30I623 unit has : 91668 rows


In [40]:
df

datetime_wita,heartbeat,unitno,deviceid,gpsspeed,gpsnumsat,VehicleSpeed,speedsource
datetime[μs],i64,str,str,f64,i64,i64,str
2025-12-10 08:00:00,1765332618,"""LD783""","""SLS30I623""",0.813028,6,1,"""GPS"""
2025-12-10 08:00:00,1765332619,"""LD783""","""SLS30I623""",0.6482,6,1,"""GPS"""
2025-12-10 08:00:00,1765332621,"""LD783""","""SLS30I623""",0.56486,7,1,"""GPS"""
2025-12-10 08:00:00,1765332634,"""LD783""","""SLS30I623""",1.263064,7,1,"""GPS"""
2025-12-10 08:00:00,1765332635,"""LD783""","""SLS30I623""",1.263064,7,1,"""GPS"""
…,…,…,…,…,…,…,…
2025-12-14 08:00:00,1765673940,"""LD783""","""SLS30I623""",3.765116,9,4,"""CANBus"""
2025-12-14 08:00:00,1765673941,"""LD783""","""SLS30I623""",3.765116,9,4,"""CANBus"""
2025-12-14 08:00:00,1765673942,"""LD783""","""SLS30I623""",3.679924,9,3,"""CANBus"""


In [ ]:
test_str = [
    "datalog/BRCG/SLS30I614/2025121212/2025121212.txt.gz",
    "datalog/BRCG/SLS30I614/2025121212/2025121212.txt.gz",
]

In [ ]:
bucket_name = "smartdbucket"
s3key_list_string = (
    f"['s3://{bucket_name}/" + "', 's3://{bucket_name}/".join(test_str) + "']"
)
s3key_list_string

"['s3://smartdbucket/datalog/BRCG/SLS30I614/2025121212/2025121212.txt.gz', 'datalog/BRCG/SLS30I614/2025121212/2025121212.txt.gz']"

In [ ]:
r = list(range(0, 24))
r = [f"{i:02d}" for i in r]
r

['00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23']

In [ ]:
main_df.select(pl.col("speedsource")).unique()

In [ ]:
s3 = boto3.client(
    "s3",
    aws_access_key_id=storage_options["aws_access_key_id"],
    aws_secret_access_key=storage_options["aws_secret_access_key"],
    region_name=storage_options["aws_region"],
)
response = s3.list_objects_v2(
    Bucket="smartdbucket", Prefix="datalog/BRCB/SDLIR002/", MaxKeys=10
)

print(f"KeyCount: {response.get('KeyCount', 0)}")
print(f"Contents: {response.get('Contents', [])}")

KeyCount: 10
Contents: [{'Key': 'datalog/BRCB/SDLIR002/2025073002/2025073002.txt.gz', 'LastModified': datetime.datetime(2025, 8, 6, 10, 1, 10, tzinfo=tzutc()), 'ETag': '"e042e6cb244ef3fe324fc34527cf355a"', 'ChecksumAlgorithm': ['CRC32'], 'ChecksumType': 'FULL_OBJECT', 'Size': 353587, 'StorageClass': 'STANDARD'}, {'Key': 'datalog/BRCB/SDLIR002/2025073003/2025073003.txt.gz', 'LastModified': datetime.datetime(2025, 8, 6, 10, 1, 12, tzinfo=tzutc()), 'ETag': '"b828e44ec0fcd26098555011b49381f9"', 'ChecksumAlgorithm': ['CRC32'], 'ChecksumType': 'FULL_OBJECT', 'Size': 192000, 'StorageClass': 'STANDARD'}, {'Key': 'datalog/BRCB/SDLIR002/2025073006/2025073006.txt.gz', 'LastModified': datetime.datetime(2025, 8, 6, 10, 1, 17, tzinfo=tzutc()), 'ETag': '"c2768535c18a4123e2c03846146b6147"', 'ChecksumAlgorithm': ['CRC32'], 'ChecksumType': 'FULL_OBJECT', 'Size': 224416, 'StorageClass': 'STANDARD'}, {'Key': 'datalog/BRCB/SDLIR002/2025073007/2025073007.txt.gz', 'LastModified': datetime.datetime(2025, 8, 6

In [ ]:
main_df.group_by(pl.col(pl.col('speedsource'))

In [ ]:
# pull gzip data from s3
s3 = s3fs.S3FileSystem(
    key=storage_options["aws_access_key_id"],
    secret=storage_options["aws_secret_access_key"],
)


filepath = f"s3://{storage_options['aws_bucket']}/datalog/{distrik}/{deviceid}/{hivehour}/{hivehour}"
bucketpath


In [ ]:
bucketpath = f"s3://{storage_options['aws_bucket']}/datalog/{distrik}/{deviceid}/{hivehour}/{hivehour}.txt.gz"
df = conn.sql(f"""
SELECT CAST(to_timestamp(heartbeat) as DATE)+ INTERVAL 8 HOUR as datetime_wita,gpsspeed,gpsnumsat,vehiclespeed,speedsource,
FROM read_json_auto('{bucketpath}')
WHERE gpsspeed > 0 AND VehicleSpeed > 0 AND speedsource = 'CANBus

           """).pl()

In [ ]:
df.filter(pl.col("speedsource") == "CANBus")

In [ ]:
px.